### ELT Project - Analysis Section

#### Table of Contents:
1. Database Neighbors
2. Current Predicted Activity
3. Age Over Expected
4. Top Passwords
5. Name Length by Country

In [1]:
import duckdb
import pandas as pd
import ipykernel

In [2]:
# connect to duckdb
con = duckdb.connect("../transform/restore/random_people.duckdb")

# print all columns of pandas df
pd.set_option('display.max_columns', None)

## DB Neighbors
### This section explores people in my database who share key fields, or "neighbor" attributes.
### I selected **City/State**, **First Name**, **Last Name**, **Birthdate**, and **Password** as matching criteria.
### I used the LISTAGG function to aggregate all individuals who share values for each attribute into similar clusters of similarity.
## Summary:
### There is **1 match** with the same city/state, **11 matches** with a shared first name, **24 matches** with a shared last name, **2 matches** with a shared birthdate (year/month/day), and **4 matches** with a shared password.

In [3]:
neighbors_query = """
with city_state_neighbors as (
    select 
        concat(address_city, ', ', address_state) as city_state,
        'City/State' as neighbor_type,
        listagg(full_name, ', ') as neighbor_list,
        listagg(distinct concat(address_city, ', ', address_state), ', ') as shared_value,
        count(*) as neighbor_count
    from all_people 
    group by 1
    having neighbor_count > 1
),

first_name as (
    select 
        first_name,
        'First Name' as neighbor_type,
        listagg(full_name, ', ') as neighbor_list,
        listagg(distinct first_name, ', ') as shared_value,
        count(*) as neighbor_count
    from all_people 
    group by 1
    having neighbor_count > 1
),

last_name as (
    select 
        last_name,
        'Last Name' as neighbor_type,
        listagg(full_name, ', ') as neighbor_list,
        listagg(distinct last_name, ', ') as shared_value,
        count(*) as neighbor_count
    from all_people 
    group by 1
    having neighbor_count > 1
),

birthdate as (
    select 
        dob,
        'Birthdate' as neighbor_type,
        listagg(full_name, ', ') as neighbor_list,
        listagg(distinct dob, ', ') as shared_value,
        count(*) as neighbor_count
    from all_people 
    group by 1
    having neighbor_count > 1
),

password as (
    select 
        password,
        'Password' as neighbor_type,
        listagg(full_name, ', ') as neighbor_list,
        listagg(distinct password, ', ') as shared_value,
        count(*) as neighbor_count
    from all_people 
    group by 1
    having neighbor_count > 1
)

select neighbor_type, shared_value, neighbor_list, neighbor_count from city_state_neighbors
union all
select neighbor_type, shared_value,  neighbor_list, neighbor_count from first_name
union all
select neighbor_type, shared_value,  neighbor_list, neighbor_count from last_name
union all
select neighbor_type, shared_value,  neighbor_list, neighbor_count from birthdate
union all
select neighbor_type, shared_value,  neighbor_list, neighbor_count from password
"""

neighbors_query = con.execute(neighbors_query).df()
neighbors_query

,neighbor_type,shared_value,neighbor_list,neighbor_count
0,City/State,"Nelson, Wellington","Madison Walker, Evelyn Kumar",2
1,First Name,Tristan,"Tristan Carroll, Tristan Abraham",2
2,First Name,Lou,"Lou Petit, Lou Louis",2
3,First Name,Silke,"Silke Jørgensen, Silke Jørgensen",2
4,First Name,Daniel,"Daniel Mitchell, Daniel Nielsen",2
5,First Name,Vsevolod,"Vsevolod Ishchenko, Vsevolod Chuprina",2
6,First Name,David,"David Lozano, David Davies",2
7,First Name,مریم,"مریم حسینی, مریم پارسا",2
8,First Name,Endre,"Endre Nordgård, Endre Malmo",2
9,First Name,Emily,"Emily Andersen, Emily Petersen",2


## Current Predicted Activity
#### Uses the timezone offset fields from the random people API to add/subtract time to the current timestamp at run time to reasonably estimate what each person in my database is doing at the moment. Results below are from my last run, but will vary dependent on run time. I did not add logic for weekends.

#### At the moment...
#### 40 doing leisure activites, 60 working, 94 sleeping, 15 commuting to work, 33 eating breakfast, 9 are eating lunch 

In [4]:
person_activities_query = """
with person_activities as (
    select
    uuid,
    timezone_offset,
    current_localtime() + cast(hour_tz_adjustment as int) * interval 1 HOUR + cast(minute_tz_adjustment as int) * interval 1 MINUTE as person_local_timestamp,
    substring(cast(person_local_timestamp as varchar), 1, 5) as local_time,
    case 
        when local_time <= '06:00' then 'Sleeping'
        when local_time <= '08:00' then 'Breakfast'
        when local_time <= '09:00' then 'Work Commute'
        when local_time <= '12:00' then 'Working'
        when local_time <= '13:00' then 'Lunch'
        when local_time <= '17:00' then 'Working'
        when local_time <= '18:00' then 'Work Commute'
        when local_time <= '23:00' then 'Leisure'
        when local_time <= '24:00' then 'Sleeping'
        else null
        end as predicted_current_activity
    from all_people)

select predicted_current_activity, count(*) as num_doing_activitiy
from person_activities
group by 1
order by 2 desc
"""

person_activities_query = con.execute(person_activities_query).df()
person_activities_query

,predicted_current_activity,num_doing_activitiy
0,Sleeping,94
1,Working,60
2,Leisure,40
3,Breakfast,33
4,Work Commute,15
5,Lunch,9


## Age Over Expected
#### **"Jeanette's"** in my database are younger than you'd expect (45 years under expected)
#### **"Rouissa's"** are older (39 years over expected)

In [19]:
age_deltas_query = """
select 
    first_name,
    avg(age) as avg_age, 
    avg(predicted_age) as agify_predicted_age, 
    count(*) as people_in_group,
    avg_age - agify_predicted_age as age_delta
from all_people 
where predicted_age is not null
group by 1
order by age_delta
"""
 
age_deltas_query = con.execute(age_deltas_query).df()
age_deltas_query

,first_name,avg_age,agify_predicted_age,people_in_group,age_delta
0,Jeanette,26.0,71.0,1,-45.0
1,Sylviane,26.0,71.0,1,-45.0
2,Rosina,28.0,72.0,1,-44.0
3,Barbara,33.0,74.0,1,-41.0
4,Irmgard,37.0,75.0,1,-38.0
...,...,...,...,...,...
177,Niklas,70.0,34.0,1,36.0
178,Shrishti,68.0,31.0,1,37.0
179,Minttu,72.0,33.0,1,39.0
180,Rafayil,79.0,40.0,1,39.0


## Top Passwords
##### I scored the password strength of everyone in my duckdb database. The scoring system used is rather basic:
##### 1 point for uppercase character, 1 point for lowercase character, 1 point for number, 1 point for a password longer than 7 characters

#### While not great passwords in a traditional sense, the highest scoring passwords were:
## porsche9, 1x2zkg8w, cricket1, 1a2b3c4d, thunder1

In [20]:
top_passwords_query = """ 
select 
    full_name, 
    password 
from all_people 
where password_complexity_score = (select max(password_complexity_score) from all_people)
order by password_complexity_score desc
"""

top_passwords_query = con.execute(top_passwords_query).df()
top_passwords_query

,full_name,password
0,Santana Pinto,cricket1
1,Iiris Ramo,1a2b3c4d
2,هلیا قاسمی,thunder1
3,ثنا موسوی,porsche9
4,Anisa Denis,1x2zkg8w


## Longest and Shortest First Names by Country (based on character count)
## Longest:
#### Mexico 🇲🇽 (7.3), Ukraine 🇺🇦 (7.1), Spain 🇪🇸 (6.9)

## Shortest:
#### USA 🇺🇸 (5.2), Turkey 🇹🇷 (5.1), Iran 🇮🇷 (4.9)

In [21]:
name_length_query = """
with name_length as (
    select
        address_country, 
        avg(len(first_name)) as average_name_length,
        row_number() over (order by average_name_length desc) as longest_names, 
        row_number() over (order by average_name_length) as shortest_names 
    from people_mart 
    group by 1 
    order by 2 desc
)

select
    address_country, 
    average_name_length 
from name_length 
where longest_names <= 3 or shortest_names <= 3
"""

name_length_query = con.execute(name_length_query).df()
name_length_query

,address_country,average_name_length
0,Mexico,7.285714
1,Ukraine,7.125000
2,Spain,6.866667
3,United States,5.200000
4,Turkey,5.071429
5,Iran,4.846154
